In [36]:
# Insert code here.
import pandas as pd
import numpy as np
import random
import re
import time
import datetime
import math
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, neighbors
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, BertConfig, AutoModel
import torch
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification
from transformers import get_linear_schedule_with_warmup
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from torch.utils.data import Dataset
from tqdm import tqdm
import pytorch_lightning as pl
import neptune
import pickle
from collections import defaultdict as dd
# with open("token", "r") as f:
#     token = f.read()
# neptune.init(project_qualified_name='tathagataraha/contro-base',
#              api_token=token,
#              )
# from sentence_transformers import SentenceTransformer
# sent_encoder = SentenceTransformer('bert-base-nli-mean-tokens')

ERROR! Session/line number was not unique in database. History logging moved to new session 795


In [37]:
with open('../data/sent_scores.pickle', 'rb') as f:
    tmp = pickle.load(f)

In [38]:
# sent = dd(0.5)
# for i in tmp:
#     sent[i[0]] = i[1]

In [39]:
sentsum = 0
for i in tmp:
    sentsum += i[1]
sentavg = sentsum/(len(tmp))
sent = dd(lambda: sentavg)
sent_scores = dd(lambda: sentsum/(len(tmp)))
for i in tmp:
    sent[i[0]] = i[1]

In [40]:
for i in sent:
    if sent[i]>1000:
        print(sent[i])

In [41]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda:0")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
torch.cuda.empty_cache()

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1080 Ti


In [42]:
models = ['bert-base-uncased','roberta-base', 'vinai/bertweet-base']
labels = ['is_cont', 'humor_rating', 'humor_controversy', 'offense_rating']
model_num = 1
label_num = 0

In [43]:
params = {
    'model' : models[model_num],
    'label' : labels[label_num],
    'valid_size' : 0.2,
    'rnd' : 42,
    'max_len' : 64,
    'train_batch' : 32,
    'valid_batch' : 32,
    'epochs' : 4,
    'lr' : 1e-05,
    'dropout' : 0.1,
    'pers' : True,
    'sent': False
}

In [44]:
params

{'model': 'roberta-base',
 'label': 'is_cont',
 'valid_size': 0.2,
 'rnd': 42,
 'max_len': 64,
 'train_batch': 32,
 'valid_batch': 32,
 'epochs': 4,
 'lr': 1e-05,
 'dropout': 0.1,
 'pers': True,
 'sent': False}

In [45]:

tokenizer = AutoTokenizer.from_pretrained(params['model'])

In [46]:
# train = pd.read_csv('data/dataset.csv')
import pickle
with open('../data/complete.pickle', 'rb') as f:
    data = pickle.load(f)
    
with open('../data/perspective_complete.pickle','rb') as f:
    pers_scores = pickle.load(f)
    print(pers_scores.keys())
# valid = pd.read_csv('data/public_dev.csv')
# train = pd.concat([train, valid])

dict_keys(['TOXICITY_WHOLE', 'TOXICITY_RAW', 'SEVERE_TOXICITY_WHOLE', 'SEVERE_TOXICITY_RAW', 'IDENTITY_ATTACK_WHOLE', 'IDENTITY_ATTACK_RAW', 'INSULT_WHOLE', 'INSULT_RAW', 'PROFANITY_WHOLE', 'PROFANITY_RAW', 'THREAT_WHOLE', 'THREAT_RAW', 'SEXUALLY_EXPLICIT_WHOLE', 'SEXUALLY_EXPLICIT_RAW', 'OBSCENE_WHOLE', 'OBSCENE_RAW'])


In [47]:
train=pd.DataFrame.from_dict(data).rename(columns = {'tweet_raw_text':'text', 'task_1':'is_cont'})

In [48]:
df = train

In [49]:
train.shape, list(pers_scores['TOXICITY_WHOLE'].keys())[-1]

((86975, 16), 80765)

In [50]:
parents = df[df['parent_ID'] == '-1']
comments = df[df['parent_ID'] != '-1']
pdict = parents.reset_index(drop=True).to_dict()
new = dd()
values = dd()
for i in range(len(pdict['tweet_id'])):
    parent = pdict['tweet_id'][i]
    tmp = comments[comments['parent_ID']==parent]
    new[parent] = tmp.text.values
    values[parent] = pdict['is_cont'][i]
#     tmp1.append(tmp.shape[0])
#     break
ones = 0
zeros = 0
for i in values.keys():
    if values[i] == '1':
        ones += 1
    else:
        zeros += 1
ones, zeros

(1353, 1323)

In [51]:
count = 0
counts = []
for i in new.keys():
    tmp = []
    for j in new[i]:
        if '&gt' not in j or j.strip() != '':
            tmp.append([count, j])
            count += 1
    counts.append(len(tmp))
    new[i] = tmp
#     print(new[i])
#     break
count

80772

In [52]:
pdict['tweet_id'][1053], pdict.keys(), new[pdict['tweet_id'][1053]]

('1369058597699809291',
 dict_keys(['tweet_id', 'is_cont', 'full_tweet', 'text', 'hashtags', 'smiley', 'emoji', 'url', 'mentions', 'numerals', 'reserved_word', 'emotext', 'segmented_hash', 'user_ID', 'root_ID', 'parent_ID']),
 [[31468,
   "Me wondering if 'good lad' Andrew is a 'exalted blessing' according to Morgan"],
  [31469,
   'if Diana was alive today, Markle would not have got in the palace door!'],
  [31470, 'The truth hurts'],
  [31471,
   'Is that the the great friend of Donald Trump and regular attendee at parties thrown by Jeffery Epstein???'],
  [31472,
   'Who cares what Piers Morgan thinks... Prince Harry &amp; Meghan spoke their minds and truth about the Monarchy...and how they were feeling. They are both human beings and deserve a life of freedom.'],
  [31473,
   'why should I care what Piers thinks? I have my own opinion. I believe Meghan and Harry.'],
  [31474,
   "Piers Morgan is a bully and a horrible person and hopefully soon we won't have to see the horrible litt

In [53]:
cnt = 0
for i in range(len(pdict['tweet_id'])):
    sum_sent = 0
    for j in new[pdict['tweet_id'][i]]:
        try:
            sum_sent += sent[j[0]]
        except:
            sum_sent += sentavg
#         print(sent[j[0]])
    sent_scores[i] = sum_sent/len(new[pdict['tweet_id'][i]])
#     print(sum_sent, len(new[pdict['tweet_id'][i]]))
    cnt += 1
#     if cnt == 10: 
#         break

In [54]:
sent_scores[0]

0.22081698008114473

In [55]:
from collections import defaultdict as dd
def dflt():
    return -1
deleted = []
for i in pers_scores.keys():
    if len(pers_scores[i]) == 0:
        deleted.append(i)
    else:
        pers_scores[i] = dd(dflt, pers_scores[i])
for i in deleted:
    pers_scores.pop(i)
print(pers_scores.keys())
final_scores = {
    'ids' : [],
    'targets' : [],
    'scores' : [],
    'parent' : [],
    'sent' : []
}
prnt_cnt = 0
for i in new.keys():
    comments_score = []
    cmt_count = 0
    for j in new[i]:
        if pers_scores['TOXICITY_WHOLE'][j[0]]!= -1:
            cmt_count += 1
            tmp1 = []
            for k in pers_scores.keys():
                tmp1.append(pers_scores[k][j[0]])
            comments_score.append(tmp1)
    comments_score = np.array(comments_score)
    final_scores['ids'].append(i)
    final_scores['targets'].append(values[i])
    mn = np.mean(comments_score, axis=0)
    if mn.shape!=(8,):
        mn = np.array([0] * 8)
    final_scores['scores'].append(mn)
    final_scores['sent'].append(sent_scores[prnt_cnt])
    final_scores['parent'].append(pdict['text'][prnt_cnt])
    prnt_cnt += 1
#     break

dict_keys(['TOXICITY_WHOLE', 'SEVERE_TOXICITY_WHOLE', 'IDENTITY_ATTACK_WHOLE', 'INSULT_WHOLE', 'PROFANITY_WHOLE', 'THREAT_WHOLE', 'SEXUALLY_EXPLICIT_WHOLE', 'OBSCENE_WHOLE'])


/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [56]:
train = pd.DataFrame(final_scores)
train

,ids,targets,scores,parent,sent
0,1376213721568006145,1,"[0.1221369183, 0.0559302568, 0.0720628692, 0.0...",Friendship is Always Better Than Relationship ...,0.220817
1,1376208370135273475,1,"[0.16244625920689656, 0.0748024928275862, 0.09...",Friendship is Always Better Than Relationship ...,0.202802
2,1376190866130280461,1,"[0.27947397323529405, 0.1383523302352941, 0.17...",I heard frm a cousin dat famly membrs r mockin...,0.197203
3,1376188572022153217,1,"[0.3174494186153846, 0.1685325290769231, 0.193...",Please Report and Block This PR Bahan Time to ...,0.149427
4,1376173435785601029,1,"[0.2449906602, 0.11239089075, 0.1557814657, 0....",Antilia case is also a high profile case with ...,0.179665
...,...,...,...,...,...
2671,1378623262486372355,0,"[0.2042153005, 0.11953716121428573, 0.13868204...",Mukhtar Ansari did not eat yesterday night. He...,0.544267
2672,1378616670155988998,0,"[0.1855257770555555, 0.11055643825000003, 0.12...",BSP leader will be lodged in barrack number of...,0.153426
2673,1378616226427981827,0,"[0.26946719613636366, 0.18065712163636372, 0.1...",Pic : How we see UP Police. Pic : How gangster...,0.202820
2674,1378616121046097920,0,"[0.2062484501818182, 0.1309215622727273, 0.144...",Why are Punjab police do gently wheeling Mukht...,0.439323


In [57]:
final_scores['scores'][1053], new[final_scores['ids'][1053]]

(array([0.31793552, 0.1926329 , 0.19398184, 0.3156712 , 0.17824667,
        0.23254247, 0.10227367, 0.15558852]),
 [[31468,
   "Me wondering if 'good lad' Andrew is a 'exalted blessing' according to Morgan"],
  [31469,
   'if Diana was alive today, Markle would not have got in the palace door!'],
  [31470, 'The truth hurts'],
  [31471,
   'Is that the the great friend of Donald Trump and regular attendee at parties thrown by Jeffery Epstein???'],
  [31472,
   'Who cares what Piers Morgan thinks... Prince Harry &amp; Meghan spoke their minds and truth about the Monarchy...and how they were feeling. They are both human beings and deserve a life of freedom.'],
  [31473,
   'why should I care what Piers thinks? I have my own opinion. I believe Meghan and Harry.'],
  [31474,
   "Piers Morgan is a bully and a horrible person and hopefully soon we won't have to see the horrible little man on tv anymore"],
  [31475,
   'Hes right, a disgraceful betrayal by the Royal family and palace establish

In [58]:
# from sklearn.model_selection import train_test_split
# # train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train['tweet'], train['label'])
# train_x, valid_x, train_y, valid_y = train_test_split(train.text, train[params['label']], test_size=params['valid_size'], random_state=params['rnd'])

In [59]:
def count_words(text):
    try:
        return len(text.split())
    except:
        print(text)
        return None

In [60]:
# total = 0
# maxw = 0
# large_count = 0
# for i in train.text:
#     temp = count_words(i)
#     total += temp
#     maxw = temp if temp > maxw else maxw
#     large_count += 1 if temp > 64 else 0
# total/len(train.text), maxw, large_count, len(train.text)

In [61]:
# def to_int(i):
#     return int(i)
# train.is_cont = train.is_cont.apply(to_int)

In [62]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len, t = False):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.parent
        self.pers_scores = dataframe.scores
        self.sent_scores = dataframe.sent
#         self.emoji = dataframe.emoji
#         self.hash = dataframe.segmented_hash
        self.t = t
        if not self.t:
            self.targets = self.data.targets
        self.max_len = max_len
#         print(self.targets)
    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        pers = self.pers_scores[index]
        sent = self.sent_scores[index]
        text = " ".join(text.split())
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_attention_mask = True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
#         h_text = self.hash[index]
#         h_text = " ".join(h_text)
#         inputs = self.tokenizer.encode_plus(
#             h_text,
#             None,
#             truncation=True,
#             add_special_tokens=True,
#             max_length=self.max_len,
#             pad_to_max_length=True,
#             return_attention_mask = True,
#             return_token_type_ids=True
#         )
#         h_ids = inputs['input_ids']
#         h_mask = inputs['attention_mask']
#         h_token_type_ids = inputs["token_type_ids"]
#         h_inputs
#         emoji = getEmojiEmbeddings(self.emoji[index])
        if self.t:
            return {
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(mask, dtype=torch.long),
                'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
                'pers_scores': torch.tensor(pers, dtype=torch.float),
                'sent' : torch.tensor(sent, dtype=torch.float),
#                 'h_ids': torch.tensor(h_ids, dtype=torch.long),
#                 'h_mask': torch.tensor(h_mask, dtype=torch.long),
#                 'h_token_type_ids': torch.tensor(h_token_type_ids, dtype=torch.long),
#                 'emoji' : torch.tensor(emoji, dtype=torch.long),
            }
        else:
            return {
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(mask, dtype=torch.long),
                'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
                'pers_scores': torch.tensor(pers, dtype=torch.float),
                'sent_scores' : torch.tensor(sent, dtype=torch.float).reshape([-1]),
#                 'h_ids': torch.tensor(h_ids, dtype=torch.long),
#                 'h_mask': torch.tensor(h_mask, dtype=torch.long),
#                 'h_token_type_ids': torch.tensor(h_token_type_ids, dtype=torch.long),
#                 'emoji' : torch.tensor(emoji, dtype=torch.long),
                'targets': torch.tensor(int(self.targets[index]), dtype=torch.long)
            }

In [63]:
# Creating the dataset and dataloader for the neural network
# train_size = 0.85
train_data=train.sample(frac=1 - params['valid_size'],random_state=params['rnd'])
test_data=train.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)
# train_data, test_data = train_test_split(train, test_size=params['valid_size'])

test_data=test_data.reset_index(drop=True)
train_data = train_data.reset_index(drop=True)
def intify(x):
    return int(x)
print("FULL Dataset: {}".format(train.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))
test_real = test_data.targets.apply(intify).values
training_set = MultiLabelDataset(train_data, tokenizer, params['max_len'])
testing_set = MultiLabelDataset(test_data, tokenizer, params['max_len'])

FULL Dataset: (2676, 5)
TRAIN Dataset: (2141, 5)
TEST Dataset: (535, 5)


In [64]:

for i in train_data['scores'].values:
    if i.shape != (8,):
        print(i)

In [65]:
np.array([0]*8)

array([0, 0, 0, 0, 0, 0, 0, 0])

In [66]:
train_params = {'batch_size': params['train_batch'],
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': params['valid_batch'],
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [67]:
it = iter(training_loader)
while True:
    try:
        hel = next(it)
    except:
        print(hel)
        break
    print(hel['ids'].shape, hel['pers_scores'].shape, hel['sent_scores'].shape)
#     print(hel['ids'].shape)
#     break

torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) tor

/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2136: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) torch.Size([32, 8]) torch.Size([32, 1])
torch.Size([32, 64]) tor

In [68]:
class LMModelClassifier(pl.LightningModule):
    def __init__(self, params):
        super().__init__()
        self.save_hyperparameters()
        self.l1 = AutoModel.from_pretrained(params['model'])
        self.pre_classifier_1 = torch.nn.Linear(768, 768)
        self.final_layer_dim = 768
        if params['pers']:
            self.pre_classifier_2 = torch.nn.Linear(8, 8)
            self.final_layer_dim += 8
        if params['sent']:
#             self.pre_classifier_3 = torch.nn.Linear(8, 8)
            self.final_layer_dim += 1
        self.pre_classifier_x = torch.nn.Linear(self.final_layer_dim, self.final_layer_dim)
        self.dropout = torch.nn.Dropout(params['dropout'])
        self.total_loss = 0
        self.batch_count = 0
        self.epoch = 0
        self.classifier = torch.nn.Linear(self.final_layer_dim, 2)
        self.preds = []
        self.targets = []
        self.test_preds = []

    def forward(self, input_ids, attention_mask, token_type_ids, pers, sent):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state_1 = output_1[0]
        pooler_1 = hidden_state_1[:, 0]
        pooler_1 = self.pre_classifier_1(pooler_1)
        pooler_1 = torch.nn.Tanh()(pooler_1)
        pooler_1 = self.dropout(pooler_1)
#         pre_final1 = self.classifier(pooler_1)
        if params['pers']:
            pooler_2 = self.pre_classifier_2(pers)
            pooler_2 = torch.nn.Tanh()(pooler_2)
            pooler_2 = self.dropout(pooler_2)
            pooler_1 = torch.cat((pooler_1, pooler_2), 1)
        if params['sent']:
            pooler_1 = torch.cat((pooler_1, sent), 1)
        if params['sent'] or params['pers']:
            pooler_1 = self.pre_classifier_x(pooler_1)
            pooler_1 = torch.nn.Tanh()(pooler_1)
            pooler_1 = self.dropout(pooler_1)
        output = self.classifier(pooler_1)
        return output
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(params =  self.parameters(), lr=params['lr'])
        return optimizer

    def training_step(self, batch, batch_nb):
        ids = batch['ids']
        mask = batch['mask']
        token_type_ids = batch['token_type_ids']
        pers = batch['pers_scores']
        sent = batch['sent_scores']
        targets = batch['targets']
        outputs = self.forward(ids, mask, token_type_ids, pers, sent)
        loss = torch.nn.CrossEntropyLoss()(outputs, targets)
        self.total_loss += loss
        self.batch_count += 1
        logger_logs = {'training_loss': loss}
        logger_logs = {'losses': logger_logs} # optional (MUST ALL BE TENSORS)
        output = {
            'loss': loss, # required
            'progress_bar': {'training_loss': loss}, # optional (MUST ALL BE TENSORS)
            'log': logger_logs
        }
        # return a dict
        return output
    
    def on_epoch_end(self):
        self.epoch += 1
        print(f'Epoch: {self.epoch}, Loss:  {self.total_loss/self.batch_count}')
        self.total_loss=0
        self.batch_count=0
    
    def validation_step(self, batch, batch_idx):
        ids = batch['ids']
        mask = batch['mask']
        token_type_ids = batch['token_type_ids']
        pers = batch['pers_scores']
        sent = batch['sent_scores']
        targets = batch['targets']
        outputs = self.forward(ids, mask, token_type_ids, pers, sent)
        loss = torch.nn.CrossEntropyLoss()(outputs, targets)
        labels_hat = torch.argmax(outputs, dim=1)
        self.preds.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
        self.targets.extend(targets.cpu().detach().numpy().tolist())
        val_acc = torch.sum(targets == labels_hat).item() / (len(targets) * 1.0)
        output = {
            'val_loss': loss,
            'val_acc': torch.tensor(val_acc), # everything must be a tensor
        }
        return output
    
    def validation_epoch_end(self, validation_step_outputs):
        self.preds = list(np.argmax(np.array(self.preds), axis=1).flatten())
        print(classification_report(self.targets, self.preds, digits=4))
        self.preds = []
        self.targets = []
    
    def test_step(self, batch, batch_idx):
        ids = batch['ids']
        mask = batch['mask']
        token_type_ids = batch['token_type_ids']
        pers = batch['pers_scores']
        outputs = self.forward(ids, mask, token_type_ids, pers)
        labels_hat = torch.argmax(outputs, dim=1)
        self.test_preds.extend(labels_hat.cpu().detach().numpy().tolist())
        

In [69]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath='/scratch/tr/',
    filename=params['model']+'-{epoch}',
    save_top_k = -1
)


In [70]:
model = LMModelClassifier(params)
trainer = pl.Trainer(max_epochs=params['epochs'], callbacks=[checkpoint_callback], gpus=1)


GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO:lightning:TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [71]:
trainer.fit(model, train_dataloader=training_loader, val_dataloaders=testing_loader)

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.

  | Name             | Type         | Params
--------------------------------------------------
0 | l1               | RobertaModel | 124 M 
1 | pre_classifier_1 | Linear       | 590 K 
2 | pre_classifier_2 | Linear       | 72    
3 | pre_classifier_x | Linear       | 602 K 
4 | dropout          | Dropout      | 0     
5 | classifier       | Linear       | 1.6 K 
--------------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
INFO:lightning:
  | Name             | Type         | Params
--------------------------------------------------
0 | l1               | RobertaModel | 124 M 
1 | pre_classifier_1 | Linear       | 590 K 
2 | pre_classifier_2 | Linear       | 72    
3 | pre_classifier_x | Linear       | 602 K 
4 | dropout          | Dropout      | 0     
5 | classifier       | Linear       | 1.6 K 
--------------------------------------------------
1

Validation sanity check: |          | 0/? [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0     0.7969    1.0000    0.8870        51
           1     0.0000    0.0000    0.0000        13

    accuracy                         0.7969        64
   macro avg     0.3984    0.5000    0.4435        64
weighted avg     0.6350    0.7969    0.7068        64



/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

Training: |          | 0/? [00:00<?, ?it/s]

...

              precision    recall  f1-score   support

           0     0.8717    0.9203    0.8953       251
           1     0.9259    0.8803    0.9025       284

    accuracy                         0.8991       535
   macro avg     0.8988    0.9003    0.8989       535
weighted avg     0.9005    0.8991    0.8992       535

Epoch: 3, Loss:  0.20464539527893066


/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2136: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The {progress_bar:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=Tru

Validating: |          | 0/? [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0     0.8650    0.9442    0.9029       251
           1     0.9464    0.8697    0.9064       284

    accuracy                         0.9047       535
   macro avg     0.9057    0.9070    0.9046       535
weighted avg     0.9082    0.9047    0.9047       535

Epoch: 4, Loss:  0.15341147780418396


1

In [102]:
# testdf = pd.read_csv('data/final_golden.csv')
# with open('data/extracted_golden.pickle', 'rb') as f:
#     data = pickle.load(f)
# test = pd.DataFrame.from_dict(data)[['tweet_raw_text', 'task_1']].rename(columns = {'tweet_raw_text':'text', 'task_1':'is_cont'})
# test['is_cont'] = testdf['is_cont']
# test.is_cont = test.is_cont.apply(to_int)

In [103]:
# test['is_cont'] == testdf['is_cont']

In [104]:
# test = pd.read_csv('data/golden.csv')
# real = test['is_cont'].values
# test_data = test.reset_index(drop=True)
# testing = MultiLabelDataset(test_data, tokenizer, params['max_len'], t=False)
# test_params = {'batch_size': params['valid_batch'],
#                 'shuffle': False,
#                 'num_workers': 0
#                 }
# testing_loader = DataLoader(testing, **test_params)
for i in range(params['epochs']):
    model = LMModelClassifier.load_from_checkpoint('/scratch/tr/'+params['model']+'-epoch='+str(i)+'.ckpt')
    model.test_preds = []
    trainer.test(model=model, test_dataloaders=testing_loader)
    print(classification_report(test_real, model.test_preds, digits=4))

Set SLURM handle signals.
/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: |          | 0/? [00:00<?, ?it/s]

/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2136: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------
              precision    recall  f1-score   support

           0     0.8464    0.9442    0.8927       251
           1     0.9451    0.8486    0.8942       284

    accuracy                         0.8935       535
   macro avg     0.8958    0.8964    0.8935       535
weighted avg     0.8988    0.8935    0.8935       535



Set SLURM handle signals.
/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


...

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------
              precision    recall  f1-score   support

           0     0.8839    0.9402    0.9112       251
           1     0.9440    0.8908    0.9167       284

    accuracy                         0.9140       535
   macro avg     0.9140    0.9155    0.9139       535
weighted avg     0.9158    0.9140    0.9141       535



Set SLURM handle signals.
/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: |          | 0/? [00:00<?, ?it/s]

/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2136: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------
              precision    recall  f1-score   support

           0     0.8203    0.9641    0.8864       251
           1     0.9625    0.8134    0.8817       284

    accuracy                         0.8841       535
   macro avg     0.8914    0.8888    0.8841       535
weighted avg     0.8958    0.8841    0.8839       535



In [ ]:
with open('preds.pickle','wb') as f:
    pickle.dump(model.test_preds, f)

In [ ]:
model.test_preds = []
trainer.test(test_dataloaders=testing_loader)


In [ ]:
# df = pd.DataFrame()
# df['id'] = list(range(8001,9001))
# df['is_humor'] = list(model.test_preds)
# df.to_csv('preds1.csv', index=False)

In [ ]:
print(classification_report(real, model.test_preds, digits=4))
# model.test_preds == real